# Step 1: Install Flask and Flask-RESTful
You need to install Flask and Flask-RESTful libraries to work with the example. Run this in a Jupyter notebook cell:

In [1]:
!pip install Flask Flask-RESTful


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/52.8 kB ? eta -:--:--
   ---------------------------------------- 52.8/52.8 kB 2.8 MB/s eta 0:00:00


# Step 2: Code for Building a Simple REST API
You can create a basic REST API that serves data using Flask. Since Jupyter Notebooks run interactively, Flask needs to be run in the background. Here’s how you can define a simple REST API for managing a list of books:

In [5]:
from flask import Flask, jsonify, request
from flask_restful import Api, Resource
from threading import Thread

# Create a Flask app and initialize the Flask-RESTful API
app = Flask(__name__)
api = Api(app)

# Dummy data for demonstration: A list of books
books = [
    {"id": 1, "title": "1984", "author": "George Orwell"},
    {"id": 2, "title": "Brave New World", "author": "Aldous Huxley"},
    {"id": 3, "title": "Fahrenheit 451", "author": "Ray Bradbury"}
]

# Define a resource for handling books
class Book(Resource):
    def get(self, book_id=None):
        # If no book_id is provided, return the entire book list
        if book_id is None:
            return jsonify(books)
        
        # If a book_id is provided, return that book if it exists
        for book in books:
            if book["id"] == book_id:
                return jsonify(book)
        
        return {"message": "Book not found"}, 404
    
    def post(self):
        # Get new book details from the request
        new_book = request.get_json()
        books.append(new_book)
        return jsonify(new_book), 201
    
    def delete(self, book_id):
        # Delete a book with the given id
        global books
        books = [book for book in books if book["id"] != book_id]
        return {"message": "Book deleted"}, 200

# Add the resource to the API, accessible at /books and /books/<book_id>
api.add_resource(Book, '/books', '/books/<int:book_id>')

# Function to run the Flask app in the background
def run_flask():
    app.run(port=5000)

# Start Flask in a separate thread to allow Jupyter Notebook to keep running
Thread(target=run_flask).start()


This code sets up a simple RESTful API using **Flask** and **Flask-RESTful**. Here's a breakdown of the code and what it does:

### 1. **Importing necessary libraries:**
   - `Flask`: The core library for creating web applications in Python.
   - `jsonify`: A helper function to return JSON responses.
   - `request`: To access incoming HTTP request data (for example, when sending data in POST requests).
   - `Api` and `Resource`: Components of Flask-RESTful, which helps structure the API resources in an organized way.
   - `Thread`: From Python's `threading` module, used to run the Flask app in a separate thread, so that it can run in the background (useful for running in a Jupyter Notebook, for example).

### 2. **Creating the Flask app and API instance:**
   ```python
   app = Flask(__name__)
   api = Api(app)
   ```
   This initializes a Flask app and binds the Flask-RESTful `Api` to it. The `Api` instance allows easy creation of RESTful routes and resources.

### 3. **Dummy data (List of books):**
   ```python
   books = [
       {"id": 1, "title": "1984", "author": "George Orwell"},
       {"id": 2, "title": "Brave New World", "author": "Aldous Huxley"},
       {"id": 3, "title": "Fahrenheit 451", "author": "Ray Bradbury"}
   ]
   ```
   This is a list of dictionaries representing a set of books with `id`, `title`, and `author`. This data serves as the backend storage for books.

### 4. **Defining the `Book` resource:**
   The `Book` class is a RESTful resource that defines the behavior for GET, POST, and DELETE requests.

   - **GET request:**
     ```python
     def get(self, book_id=None):
     ```
     - If no `book_id` is provided, it returns the entire list of books.
     - If a specific `book_id` is provided, it searches for the corresponding book and returns it. If the book isn't found, it returns a 404 error with a "Book not found" message.

   - **POST request:**
     ```python
     def post(self):
     ```
     - It reads JSON data from the request body and appends the new book to the `books` list, then returns the newly added book with a 201 status code.

   - **DELETE request:**
     ```python
     def delete(self, book_id):
     ```
     - It searches for a book by `book_id` and removes it from the `books` list. If found and deleted, it returns a success message. Otherwise, it would still return 200 if the book doesn't exist (this logic could be expanded).

### 5. **Adding the `Book` resource to the API:**
   ```python
   api.add_resource(Book, '/books', '/books/<int:book_id>')
   ```
   This binds the `Book` resource to two endpoints:
   - `/books`: Handles requests that work on the entire book collection (e.g., GET all books or POST a new book).
   - `/books/<int:book_id>`: Handles requests that work on specific books by `book_id` (e.g., GET or DELETE a single book).

### 6. **Running the Flask app in a separate thread:**
   ```python
   def run_flask():
       app.run(port=5000)

   Thread(target=run_flask).start()
   ```
   This function starts the Flask app on port 5000, but instead of blocking the main thread (which is a problem if you are running this code in a Jupyter notebook or an environment that needs to stay interactive), it runs the Flask app in a background thread using Python's `Thread`. This allows the web server to run while other code can continue executing in the notebook or environment.

---

### **Summary of API Operations:**
- **GET /books**: Retrieve all books.
- **GET /books/<book_id>**: Retrieve a specific book by its `book_id`.
- **POST /books**: Add a new book to the list. The new book data must be sent as JSON.
- **DELETE /books/<book_id>**: Remove a specific book from the list by its `book_id`.

This API is a simple demonstration of how Flask-RESTful can be used to create a REST API, handling basic CRUD operations for a list of books.

# Step 3: Testing the REST API Endpoints
Now that the Flask API is running in the background, you can test the API endpoints using Python's requests library. Below are some examples for testing the REST API:

## 1. Get All Books (GET /books)

In [8]:
import requests

# Fetch the list of all books
response = requests.get('http://127.0.0.1:5000/books')
print(response.json())


127.0.0.1 - - [24/Oct/2024 22:06:22] "GET /books HTTP/1.1" 200 -


[{'author': 'George Orwell', 'id': 1, 'title': '1984'}, {'author': 'Aldous Huxley', 'id': 2, 'title': 'Brave New World'}, {'author': 'Ray Bradbury', 'id': 3, 'title': 'Fahrenheit 451'}]


This code uses the `requests` library in Python to interact with a REST API (specifically, the API you created in the previous Flask example). Here’s an explanation of what each part of the code does:

### 1. **Importing the `requests` library:**
   ```python
   import requests
   ```
   - The `requests` library is a powerful Python package used to send HTTP requests and interact with web servers. It makes it easy to perform various types of requests like GET, POST, PUT, DELETE, etc.

### 2. **Sending a GET request to fetch the list of all books:**
   ```python
   response = requests.get('http://127.0.0.1:5000/books')
   ```
   - **`requests.get`**: This function sends a **GET** request to the specified URL, which in this case is `http://127.0.0.1:5000/books`. This URL points to the local instance of the Flask application that serves the list of books.
   
   - The GET request is targeting the `/books` endpoint, which is defined in your Flask application to return all the books in JSON format.

   - **`http://127.0.0.1:5000`**: This is the URL of the locally running Flask server. `127.0.0.1` refers to the local machine (localhost), and `5000` is the port number where the Flask app is running.
   
   - The response from this request is stored in the `response` variable. This `response` object contains various attributes, including the status code of the HTTP request, response headers, and the actual content (data) returned by the API.

### 3. **Printing the JSON response:**
   ```python
   print(response.json())
   ```
   - **`response.json()`**: This method converts the response content (which is in JSON format) into a Python dictionary or list, depending on the structure of the JSON.
   
   - Since the `/books` endpoint returns a list of books, this will be converted into a Python list of dictionaries (each dictionary representing a book).
   
   - **`print(response.json())`**: This prints the parsed JSON data, which should be the list of books (as defined in the Flask application).

### Expected Output:
Assuming the Flask API from the earlier example is running and has the dummy data of books, the output would be something like:
```python
[
    {"id": 1, "title": "1984", "author": "George Orwell"},
    {"id": 2, "title": "Brave New World", "author": "Aldous Huxley"},
    {"id": 3, "title": "Fahrenheit 451", "author": "Ray Bradbury"}
]
```

This code fetches the data from the running Flask API and prints it in a readable JSON format.

## 2. Get a Single Book by ID (GET /books/<id>)

In [11]:
# Fetch a single book by its ID
book_id = 1
response = requests.get(f'http://127.0.0.1:5000/books/{book_id}')
print(response.json())


127.0.0.1 - - [24/Oct/2024 22:06:56] "GET /books/1 HTTP/1.1" 200 -


{'author': 'George Orwell', 'id': 1, 'title': '1984'}


In [39]:
This code fetches a single book by its `book_id` from the Flask API and prints the result in JSON format.

### Breakdown:
1. **Setting the `book_id`:**
   ```python
   book_id = 1
   ```
   - This defines the ID of the book you want to fetch from the API. In this case, it's `1`.

2. **Sending a GET request:**
   ```python
   response = requests.get(f'http://127.0.0.1:5000/books/{book_id}')
   ```
   - This sends a **GET** request to the endpoint `/books/<book_id>` of the locally running Flask server (`http://127.0.0.1:5000`).
   - The `f` in front of the string allows string interpolation, inserting `book_id` directly into the URL.

3. **Printing the JSON response:**
   ```python
   print(response.json())
   ```
   - This retrieves and prints the JSON data returned by the API, which contains the details of the book with the specified `book_id`.

### Expected Output (if `book_id = 1`):
```python
{"id": 1, "title": "1984", "author": "George Orwell"}
```

In summary, this code retrieves a specific book by its ID and prints the details in JSON format.

SyntaxError: unterminated string literal (detected at line 8) (3961017179.py, line 8)

## 3. Delete a Book (DELETE /books/<id>)

In [30]:
# Delete a book by its ID
book_id = 2
response = requests.delete(f'http://127.0.0.1:5000/books/{book_id}')
print(response.json())


127.0.0.1 - - [24/Oct/2024 22:10:41] "DELETE /books/2 HTTP/1.1" 200 -


{'message': 'Book deleted'}


This code sends a **DELETE** request to remove a book by its `book_id` from the Flask API and prints the response in JSON format.

### Breakdown:
1. **Setting the `book_id`:**
   ```python
   book_id = 2
   ```
   - Defines the ID of the book you want to delete. Here, it’s `2`.

2. **Sending a DELETE request:**
   ```python
   response = requests.delete(f'http://127.0.0.1:5000/books/{book_id}')
   ```
   - Sends a **DELETE** request to the `/books/<book_id>` endpoint on the local Flask server (`http://127.0.0.1:5000`).
   - The `f` allows string interpolation to include the `book_id` in the URL.

3. **Printing the JSON response:**
   ```python
   print(response.json())
   ```
   - Retrieves and prints the server’s response, typically a confirmation message like `"Book deleted"`.

### Expected Output (if the deletion is successful):
```python
{"message": "Book deleted"}
```

In summary, this code deletes a specific book by its ID and prints the server’s confirmation in JSON format.